In [65]:
from google.cloud import aiplatform
from google.cloud import bigquery

import pandas as pd
import numpy as np
from datetime import date,datetime,timedelta

import math
import os

import tensorflow as tf
import tensorflow_decision_forests as tfdf  # constantly registered to load model 
print(tf.__version__)
print(tfdf.__version__)

2.12.0
1.4.0


In [66]:
# import functions_framework
# @functions_framework.http
# def xgb_tf_predict_incident_severity(request):

In [76]:
option=1
_model='model_binary_xgb_tf'   # the is resonable because unsseen and tune 70&

model_gs_path=f"gs://demo-tuned-tf-incident-pongthorn/{_model}"
print(model_gs_path)

projectId="pongthorn"
dataset_id="SMartML"

prediction_table="new2_result_binary_prediction_incident"
if option==1:
    unusedCols_unseen=['id','severity_name','open_to_close_hour']
else:
    unusedCols_unseen=['id','severity_name','range_open_to_close_hour']
    

gs://demo-tuned-tf-incident-pongthorn/model_binary_xgb_tf


# Load Model

In [68]:
abc_model = tf.keras.models.load_model(model_gs_path)  
print(abc_model.summary())
# abc_model = tf.keras.models.load_model(model_local_path) 

[INFO 23-07-12 05:43:47.3937 UTC kernel.cc:1243] Loading model from path gs://demo-tuned-tf-incident-pongthorn/model_binary_xgb_tf/assets/ with prefix aa04ef13aac245e5
[INFO 23-07-12 05:43:48.0165 UTC abstract_model.cc:1311] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 23-07-12 05:43:48.0173 UTC kernel.cc:1075] Use fast generic engine


Model: "gradient_boosted_trees_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
None


# Make Prediction

In [69]:
client = bigquery.Client(project=projectId)
new_data_table_id=f"{projectId}.{dataset_id}.new2_incident"

query_result=client.query(f"SELECT * FROM {new_data_table_id}")
df=query_result.to_dataframe()

# unseen =df.drop(columns=unusedCols_unseen)
print(unseen.info())
unseen.tail(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   severity_id               80 non-null     Int64 
 1   sla                       80 non-null     object
 2   product_type              80 non-null     object
 3   brand                     80 non-null     object
 4   service_type              80 non-null     object
 5   incident_type             80 non-null     object
 6   range_open_to_close_hour  80 non-null     object
dtypes: Int64(1), object(6)
memory usage: 4.6+ KB
None


,severity_id,sla,product_type,brand,service_type,incident_type,range_open_to_close_hour
70,4,24x7 4Hrs Resolution Time,Access Point,Cisco,Request,General Incident,soonest
71,4,24x7 4Hrs Resolution Time,Access Point,Cisco,Request,General Incident,soonest
72,4,24x7 4Hrs Resolution Time,Software,Veeam,Request,Software,soonest
73,4,24x7 4Hrs Response Time,Storage,NetApp,Incident,General Incident,soon
74,4,24x7 4Hrs Resolution Time,Server,Oracle,Request,OS / Firmware,soonest
75,4,24x7 4Hrs Resolution Time,Server,Oracle,Request,Software,soonest
76,4,24x7 4Hrs Resolution Time,Server,Oracle,Request,Software,soonest
77,4,24x7 4Hrs Resolution Time,Software,VMWare,Request,Software,soonest
78,4,24x7 4Hrs Resolution Time,Software,VMWare,Request,Software,soon
79,4,24x7 4Hrs Response Time,Storage,Veritas,Incident,General Incident,soon


In [70]:
unseen_ds= tfdf.keras.pd_dataframe_to_tf_dataset(unseen.drop(columns=['severity_id']))
print(unseen_ds)

<_PrefetchDataset element_spec={'sla': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'product_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'brand': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'service_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'incident_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'range_open_to_close_hour': TensorSpec(shape=(None,), dtype=tf.string, name=None)}>


In [71]:
predResultList=abc_model.predict(unseen_ds)
predServerityIDList=[]
for predResult in predResultList:
    # prob = tf.nn.sigmoid(predResult) # no need to convert to Signmoid
    # print(prob)
    _class= 1 if predResult[0]>=0.5 else 0  
    # pred_seveirty_id= "critical" if _class==1 else "normal"
    predServerityIDList.append(_class) #0=normal , 1=critical
    print(f"{predResult} : {_class}")

dfPred=pd.DataFrame(data=predServerityIDList,columns=["pred_binary_severity"])   


1/1 [==============================] - 0s 73ms/step
[0.9034521] : 1
[0.9237812] : 1
[0.9237812] : 1
[0.9237812] : 1
[0.9237812] : 1
[0.9237812] : 1
[0.9237812] : 1
[0.9237812] : 1
[0.94812167] : 1
[0.9034521] : 1
[0.9237812] : 1
[0.9305668] : 1
[0.8499437] : 1
[0.92120075] : 1
[0.9589557] : 1
[0.8262993] : 1
[0.8262993] : 1
[0.76963425] : 1
[0.8262993] : 1
[0.20050809] : 0
[0.794408] : 1
[0.794408] : 1
[0.41731188] : 0
[0.9715124] : 1
[0.96282226] : 1
[0.8973485] : 1
[0.04830859] : 0
[0.76828104] : 1
[0.93529826] : 1
[0.93529826] : 1
[0.93529826] : 1
[0.93529826] : 1
[0.03634203] : 0
[0.03545049] : 0
[0.91997814] : 1
[0.0315658] : 0
[0.25584435] : 0
[0.44409227] : 0
[0.16887482] : 0
[0.6111559] : 1
[0.9399456] : 1
[0.6938351] : 1
[0.2724037] : 0
[0.73289806] : 1
[0.6855305] : 1
[0.48196957] : 0
[0.48196957] : 0
[0.03005761] : 0
[0.09913998] : 0
[0.04830859] : 0
[0.22983775] : 0
[0.01278833] : 0
[0.04202354] : 0
[0.00855135] : 0
[0.04202354] : 0
[0.04202354] : 0
[0.04202354] : 0
[0.0420

2023-07-12 05:43:50.458899: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [80]
	 [[{{node Placeholder/_0}}]]


In [72]:
def map_4to2_serverity(severity_id):
    if severity_id==1 or severity_id==2:
       return 1
    else:
       return 0 
df['label_binary_severity'] =df['severity_id'].apply(map_4to2_serverity)

dfPred
df=pd.concat([df,dfPred],axis=1)
df

,id,severity_id,severity_name,sla,product_type,brand,service_type,incident_type,open_to_close_hour,range_open_to_close_hour,label_binary_severity,pred_binary_severity
0,2766,2,Major,24x7 4Hrs Resolution Time,Server,HPE,Incident,General Incident,1274.750000,latest,1,1
1,3107,2,Major,24x7 4Hrs Resolution Time,Server,HPE,Incident,General Incident,8.916667,soonest,1,1
2,3105,2,Major,24x7 4Hrs Resolution Time,Server,HPE,Incident,General Incident,8.916667,soonest,1,1
3,3104,2,Major,24x7 4Hrs Resolution Time,Server,HPE,Incident,General Incident,8.916667,soonest,1,1
4,3103,2,Major,24x7 4Hrs Resolution Time,Server,HPE,Incident,General Incident,8.916667,soonest,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
75,3094,4,Cosmetic,24x7 4Hrs Resolution Time,Server,Oracle,Request,Software,4.000000,soonest,0,1
76,3093,4,Cosmetic,24x7 4Hrs Resolution Time,Server,Oracle,Request,Software,4.000000,soonest,0,1
77,3053,4,Cosmetic,24x7 4Hrs Resolution Time,Software,VMWare,Request,Software,4.000000,soonest,0,0
78,3052,4,Cosmetic,24x7 4Hrs Resolution Time,Software,VMWare,Request,Software,76.000000,soon,0,0


# Evaluate model

In [74]:
from sklearn.metrics import confusion_matrix,classification_report
className=list(set().union(list(df['pred_binary_severity'].unique()),list(df['label_binary_severity'].unique())))
print(className)
actualClass=[  f'actual-{x}' for x in  className]
predictedlClass=[  f'pred-{x}' for x in className]
y_true=list(df['label_binary_severity'])
y_pred=list(df['pred_binary_severity'])
cnf_matrix = confusion_matrix(y_true,y_pred)
cnf_matrix

# #index=actual , column=prediction
cm_df = pd.DataFrame(cnf_matrix,
                     index = actualClass, 
                     columns = predictedlClass)
print(cm_df)
print(classification_report(y_true, y_pred, labels=className))

[0, 1]
          pred-0  pred-1
actual-0      34      14
actual-1       3      29
              precision    recall  f1-score   support

           0       0.92      0.71      0.80        48
           1       0.67      0.91      0.77        32

    accuracy                           0.79        80
   macro avg       0.80      0.81      0.79        80
weighted avg       0.82      0.79      0.79        80



# Write Prediction Result to BQ

In [75]:
df=df[['id','label_binary_severity','pred_binary_severity']]
df['prediction_datetime']=datetime.now()
print(df)

      id  label_binary_severity  pred_binary_severity  \
0   2766                      1                     1   
1   3107                      1                     1   
2   3105                      1                     1   
3   3104                      1                     1   
4   3103                      1                     1   
..   ...                    ...                   ...   
75  3094                      0                     1   
76  3093                      0                     1   
77  3053                      0                     0   
78  3052                      0                     0   
79  3029                      0                     1   

          prediction_datetime  
0  2023-07-12 05:47:05.632213  
1  2023-07-12 05:47:05.632213  
2  2023-07-12 05:47:05.632213  
3  2023-07-12 05:47:05.632213  
4  2023-07-12 05:47:05.632213  
..                        ...  
75 2023-07-12 05:47:05.632213  
76 2023-07-12 05:47:05.632213  
77 2023-07-12 05:47:05.6322

/var/tmp/ipykernel_10148/3269719424.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prediction_datetime']=datetime.now()


In [77]:
predictResult_table_id=f"{projectId}.{dataset_id}.{prediction_table}"

try:
    client.get_table(predictResult_table_id)  # Make an API request.
    print("Predict Result Table {} already exists.".format(predictResult_table_id))
except Exception as ex:
    schema = [
    bigquery.SchemaField("id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("label_binary_severity", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("pred_binary_severity", "INTEGER", mode="REQUIRED"),       
    bigquery.SchemaField("prediction_datetime", "DATETIME", mode="REQUIRED") 
    ]

    table = bigquery.Table(predictResult_table_id,schema=schema)
    # table.time_partitioning = bigquery.TimePartitioning(
    # type_=bigquery.TimePartitioningType.DAY,field="prediction_item_date")
    
    table = client.create_table(table)  # Make an API request.
    
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

Created table pongthorn.SMartML.new2_result_binary_prediction_incident


In [78]:
def loadDataFrameToBQ():
    # WRITE_TRUNCATE , WRITE_APPEND
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_TRUNCATE",
        )

        job = client.load_table_from_dataframe(
            df, predictResult_table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total Prediction ML ", len(df), "Imported bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        for e in job.errors:
            print('ERROR: {}'.format(e['message']))

try:
    loadDataFrameToBQ()
except Exception as ex:
    raise ex

Total Prediction ML  80 Imported bigquery successfully


In [ ]:
# return 'All incidents has been predicted completely.'